In [1]:
import os
import gc
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings(action = 'ignore')

In [2]:
train_transaction = pd.read_csv("../dataset/train_transaction.csv")
train_identity = pd.read_csv("../dataset/train_identity.csv")

test_transaction = pd.read_csv("../dataset/test_transaction.csv")
test_identity = pd.read_csv("../dataset/test_identity.csv")

In [3]:
%run DataPrepare.ipynb

In [4]:
train_df = pd.merge(left=train_transaction,right=train_identity,how='left',on = 'TransactionID')

In [5]:
del train_transaction, train_identity
gc.collect()

7177

In [6]:
Reduce_Memory_Usage(train_df)

origin memory usage: 1959.8762512207031
reduced memory usage: 650.4761695861816
memory reduce (66.8%)
